In [15]:
import pandas as pd
import sqlite3

In [ ]:
import requests
import json
from datetime import datetime, timedelta

def get_bitcoin_data_binance():
    """Get Bitcoin price and volume data from Binance"""
    try:
        # Binance API endpoint for 24hr ticker statistics
        url = "https://api.binance.com/api/v3/ticker/24hr"
        params = {"symbol": "BTCUSDT"}
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        return {
            "price": float(data["lastPrice"]),
            "volume_24h": float(data["volume"]),  # BTC volume
            "volume_24h_usdt": float(data["quoteVolume"]),  # USDT volume
            "price_change_24h": float(data["priceChange"]),
            "price_change_percent_24h": float(data["priceChangePercent"])
        }
    
    except requests.RequestException as e:
        print(f"Error fetching data from Binance: {e}")
        return None

def get_top_crypto_prices_binance():
    """Get prices for top cryptocurrencies from Binance to calculate dominance"""
    try:
        # Get all 24hr ticker data
        url = "https://api.binance.com/api/v3/ticker/24hr"
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        
        # Major cryptocurrencies and their typical market caps
        # We'll focus on USDT pairs for major coins
        major_pairs = [
            'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'ADAUSDT', 'SOLUSDT',
            'XRPUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'MATICUSDT',
            'LINKUSDT', 'UNIUSDT', 'LTCUSDT', 'BCHUSDT', 'XLMUSDT'
        ]
        
        prices = {}
        for ticker in data:
            symbol = ticker['symbol']
            if symbol in major_pairs:
                prices[symbol] = float(ticker['lastPrice'])
        
        return prices
    
    except requests.RequestException as e:
        print(f"Error fetching crypto prices from Binance: {e}")
        return None

def get_historical_prices_binance(symbol, days_ago=1):
    """Get historical price from Binance for a specific number of days ago"""
    try:
        # Calculate timestamp for days ago
        end_time = datetime.now() - timedelta(days=days_ago)
        start_time = end_time - timedelta(hours=1)  # 1 hour window
        
        # Convert to milliseconds
        start_ms = int(start_time.timestamp() * 1000)
        end_ms = int(end_time.timestamp() * 1000)
        
        url = "https://api.binance.com/api/v3/klines"
        params = {
            'symbol': symbol,
            'interval': '1h',
            'startTime': start_ms,
            'endTime': end_ms,
            'limit': 1
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        if data:
            # Return close price (index 4 in klines data)
            return float(data[0][4])
        return None
    
    except requests.RequestException as e:
        print(f"Error fetching historical data for {symbol}: {e}")
        return None

def calculate_btc_dominance_change():
    """Calculate BTC dominance and its change using Binance data"""
    try:
        # Get current BTC price
        btc_data = get_bitcoin_data_binance()
        if not btc_data:
            return None
        
        current_btc_price = btc_data['price']
        
        # Get BTC price from 24 hours ago
        btc_price_24h_ago = get_historical_prices_binance('BTCUSDT', days_ago=1)
        if not btc_price_24h_ago:
            print("Could not get historical BTC price")
            return None
        
        # For a more accurate calculation, we would need:
        # 1. Circulating supply for each coin
        # 2. All crypto market caps
        # 
        # For demonstration, we'll use a simplified approach with major coins
        # and estimated circulating supplies (these should be updated with real data)
        
        # Approximate circulating supplies (in millions) - these are estimates
        circulating_supplies = {
            'BTCUSDT': 19.7,  # ~19.7M BTC
            'ETHUSDT': 120.3,  # ~120.3M ETH
            'BNBUSDT': 153.4,  # ~153.4M BNB
            'ADAUSDT': 35000,  # ~35B ADA
            'SOLUSDT': 400,    # ~400M SOL
            'XRPUSDT': 54000,  # ~54B XRP
            'DOGEUSDT': 141000, # ~141B DOGE
            'AVAXUSDT': 394,   # ~394M AVAX
            'MATICUSDT': 9300, # ~9.3B MATIC
            'LINKUSDT': 556,   # ~556M LINK
            'UNIUSDT': 754,    # ~754M UNI
            'LTCUSDT': 73.4,   # ~73.4M LTC
            'BCHUSDT': 19.7,   # ~19.7M BCH
            'XLMUSDT': 28400,  # ~28.4B XLM
        }
        
        # Get current prices
        current_prices = get_top_crypto_prices_binance()
        if not current_prices:
            return None
        
        # Calculate current total market cap of tracked coins
        current_total_cap = 0
        for symbol, price in current_prices.items():
            if symbol in circulating_supplies:
                market_cap = price * circulating_supplies[symbol] * 1_000_000  # Convert to actual units
                current_total_cap += market_cap
        
        # Calculate BTC market cap
        btc_market_cap = current_btc_price * circulating_supplies['BTCUSDT'] * 1_000_000
        
        # Calculate current dominance
        current_dominance = (btc_market_cap / current_total_cap) * 100
        
        # Calculate 24h ago dominance
        btc_market_cap_24h_ago = btc_price_24h_ago * circulating_supplies['BTCUSDT'] * 1_000_000
        
        # For simplicity, assume other coins' market caps changed proportionally
        # (In reality, you'd need historical prices for all coins)
        price_change_ratio = current_btc_price / btc_price_24h_ago
        estimated_total_cap_24h_ago = current_total_cap / price_change_ratio
        
        dominance_24h_ago = (btc_market_cap_24h_ago / estimated_total_cap_24h_ago) * 100
        
        # Calculate dominance change
        dominance_change = current_dominance - dominance_24h_ago
        dominance_change_percent = (dominance_change / dominance_24h_ago) * 100
        
        return {
            "current_dominance": current_dominance,
            "dominance_24h_ago": dominance_24h_ago,
            "dominance_change": dominance_change,
            "dominance_change_percent": dominance_change_percent,
            "btc_market_cap": btc_market_cap,
            "total_tracked_cap": current_total_cap
        }
    
    except Exception as e:
        print(f"Error calculating BTC dominance: {e}")
        return None

def get_bitcoin_marketcap_dominance():
    """Get Bitcoin market cap and dominance with changes from CoinGecko (free API)"""
    try:
        # CoinGecko API for market cap and dominance
        url = "https://api.coingecko.com/api/v3/global"
        
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        
        # Get Bitcoin specific data with market data
        btc_url = "https://api.coingecko.com/api/v3/coins/bitcoin"
        btc_response = requests.get(btc_url)
        btc_response.raise_for_status()
        
        btc_data = btc_response.json()
        
        # Extract market cap and changes
        market_data = btc_data["market_data"]
        
        return {
            "market_cap": market_data["market_cap"]["usd"],
            "market_cap_change_24h": market_data["market_cap_change_24h"],
            "market_cap_change_percentage_24h": market_data["market_cap_change_percentage_24h"],
            "btc_dominance": data["data"]["market_cap_percentage"]["btc"]
        }
    
    except requests.RequestException as e:
        print(f"Error fetching data from CoinGecko: {e}")
        return None

def get_comprehensive_btc_data():
    """Get comprehensive Bitcoin data including dominance changes from CoinMarketCap"""
    try:
        # CoinMarketCap API (requires free API key)
        # Sign up at https://coinmarketcap.com/api/
        
        headers = {
            'X-CMC_PRO_API_KEY': 'ac4bc071-2576-46de-8aab-718cb1d02fb6',  # Replace with your actual API key
            'Accept': 'application/json'
        }
        
        # Get Bitcoin data
        btc_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
        btc_params = {'symbol': 'BTC'}
        
        btc_response = requests.get(btc_url, headers=headers, params=btc_params)
        btc_response.raise_for_status()
        btc_data = btc_response.json()
        
        # Get global market data for dominance
        global_url = "https://pro-api.coinmarketcap.com/v1/global-metrics/quotes/latest"
        global_response = requests.get(global_url, headers=headers)
        global_response.raise_for_status()
        global_data = global_response.json()
        
        btc_info = btc_data['data']['BTC']
        global_info = global_data['data']
        
        return {
            "market_cap": btc_info['quote']['USD']['market_cap'],
            # "market_cap_change_24h": btc_info['quote']['USD']['market_cap_change_24h'],
            # "market_cap_change_percentage_24h": btc_info['quote']['USD']['market_cap_change_percentage_24h'],
            "btc_dominance": global_info['btc_dominance'],
            "btc_dominance_change_24h": global_info['btc_dominance_24h_percentage_change']
        }
    
    except requests.RequestException as e:
        print(f"Error fetching data from CoinMarketCap: {e}")
        return None

def main():
    print("Fetching Bitcoin market data...\n")
    
    # Get data from Binance
    binance_data = get_bitcoin_data_binance()
    if binance_data:
        print("=== Binance Data ===")
        print(f"Price: ${binance_data['price']:,.2f}")
        print(f"24h Volume (BTC): {binance_data['volume_24h']:,.2f}")
        print(f"24h Volume (USDT): ${binance_data['volume_24h_usdt']:,.2f}")
        print(f"24h Change: {binance_data['price_change_percent_24h']:+.2f}%")
        print()
    
    # Get market cap and dominance from CoinGecko (free)
    market_data = get_bitcoin_marketcap_dominance()
    if market_data:
        print("=== Market Data (CoinGecko) ===")
        print(f"Market Cap: ${market_data['market_cap']:,.0f}")
        if market_data['market_cap_change_24h']:
            print(f"Market Cap Change 24h: ${market_data['market_cap_change_24h']:,.0f}")
        if market_data['market_cap_change_percentage_24h']:
            print(f"Market Cap Change % 24h: {market_data['market_cap_change_percentage_24h']:+.2f}%")
        print(f"BTC Dominance: {market_data['btc_dominance']:.2f}%")
        print()
    
    # Get comprehensive data from CoinMarketCap (requires API key)
    print("=== Comprehensive Data (CoinMarketCap) ===")
    print("Note: Replace 'YOUR_API_KEY_HERE' with actual CoinMarketCap API key")
    comprehensive_data = get_comprehensive_btc_data()
    if comprehensive_data:
        print(f"Market Cap: ${comprehensive_data['market_cap']:,.0f}")
        # print(f"Market Cap Change 24h: ${comprehensive_data['market_cap_change_24h']:,.0f}")
        # print(f"Market Cap Change % 24h: {comprehensive_data['market_cap_change_percentage_24h']:+.2f}%")
        print(f"BTC Dominance: {comprehensive_data['btc_dominance']:.2f}%")
        print(f"BTC Dominance Change 24h: {comprehensive_data['btc_dominance_change_24h']:+.2f}%")
    else:
        print("CoinMarketCap data unavailable (API key required)")

if __name__ == "__main__":
    main()

Fetching Bitcoin market data...

=== Binance Data ===
Price: $108,707.53
24h Volume (BTC): 8,466.00
24h Volume (USDT): $920,508,069.32
24h Change: +0.36%

=== Market Data (CoinGecko) ===
Market Cap: $2,163,003,520,486
Market Cap Change 24h: $10,537,832,210
Market Cap Change % 24h: +0.49%
BTC Dominance: 62.85%

=== Comprehensive Data (CoinMarketCap) ===
Note: Replace 'YOUR_API_KEY_HERE' with actual CoinMarketCap API key
Market Cap: $2,161,999,086,779
BTC Dominance: 64.13%
BTC Dominance Change 24h: -0.35%


In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import sqlite3


db_path = '../db.sqlite3'
today = datetime.now().strftime('%Y-%m-%d')
analyzer = SentimentIntensityAnalyzer()

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT title, description FROM news WHERE date = ?", (today,))
rows = cursor.fetchall()
conn.close()
sentiments = []
for title, description in rows:
    text = (title or '') + ' ' + (description or '')
    score = analyzer.polarity_scores(text)
    sentiments.append(score['compound'])
avg_sentiment = sum(sentiments) / len(sentiments)

In [17]:
avg_sentiment

-0.047479999999999994